In [8]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px

In [2]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
    # idk wtf this is --> "tech-ship0"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

In [42]:
dat_2 = pd.read_pickle("two_players_data")
dat_3 = pd.read_pickle("three_players_data")
dat_4 = pd.read_pickle("four_players_data")

In [43]:
len(dat_4), len(dat_3), len(dat_2)

(5156, 2674, 7935)

In [7]:
for faction in factions:
    factions_pos = []
    for pos in range(4):
        factions_pos.append(dat_4[dat_4["pos_" + str(pos + 1) +"_faction"] == faction])
    factions_in = pd.concat(factions_pos, ignore_index=True)
    
    
    if len(factions_in) == 0:
        factions_in.head()
        raise ValueError('faction not found: ' + faction)
    
    fig = go.Figure()

    x = []
    for val in adv_techs.values():
        x.append(val)
    

    for i in range(4):
        y = []
        for tech in adv_techs.values():
            df = factions_pos[i]
            y.append(len(df[df['pos_' + str(i + 1) + '_adv_tech_taken_' + tech] == True]) / len(factions_in))
    
        fig.add_trace(go.Bar(x=x, y=y, name='pos_' + str(i+1)))


    
    fig.update_layout(barmode='stack')
    fig.update_xaxes(categoryorder='total ascending')
    fig.update_layout(title_text=faction)
    fig.write_html("./plots/adv_techs_analysis/taken_freq/" + faction + ".html")
    fig.show()

In [37]:
for faction in factions:
    
    tech_slots = [
    "tech_adv-terra", 
    "tech_adv-nav",
    "tech_adv-int",
    "tech_adv-gaia",
    "tech_adv-eco",
    "tech_adv-sci"
    ]
    
    factions_pos = []
    for pos in range(4):
        factions_pos.append(dat_4[dat_4["pos_" + str(pos + 1) +"_faction"] == faction])
    factions_in = pd.concat(factions_pos, ignore_index=True)
    
    techs = list(adv_techs.values())
    
    slot_scores = []
    for i in range(len(tech_slots) + 1):
        slot_scores.append([])
        
    game_nums = []
    for i in range(len(tech_slots) + 1):
        game_nums.append([])
    
    fig = go.Figure()
                
    for tech in adv_techs.values():
        not_taken_score = 0
        not_taken_amount = 0
        for i, slot in enumerate(tech_slots):
            taken_score = 0
            taken_amount = 0
            for pos in range(4):
                df = factions_pos[pos]
                df = df[df[slot] == tech]
                df_not_taken = df[df['pos_' + str(pos + 1) + "_adv_tech_taken_" + tech] == False]
                not_taken_score += df_not_taken['pos_' + str(pos + 1) + "_score"].mean()
                not_taken_amount += len(df_not_taken)
                df_taken = df[df['pos_' + str(pos + 1) + "_adv_tech_taken_" + tech] == True]
                taken_score += df_taken['pos_' + str(pos + 1) + "_score"].mean()
                taken_amount += len(df_taken)
            slot_scores[i].append(taken_score / 4)
            game_nums[i].append(taken_amount)
        slot_scores[-1].append(not_taken_score / 6 / 4)
        game_nums[-1].append(not_taken_amount)
        
    tech_slots.append("not_taken")
    
    for i, slot in enumerate(tech_slots):
        fig.add_trace(go.Scatter(
                x=slot_scores[i],
                y=techs,
                mode="markers",
                name=slot,
                hovertemplate =
                    '<br><b>Position</b>: ' + slot + 
                    '<br><b>Tech</b>: %{y}' +
                    '<br><b>Ave Score</b>: %{x}' +
                    '<br><b>Num Found</b>: %{text}',
                    text = game_nums[i],
        ))
    
            
    
            
    fig.update_layout(title="Score for Advanced Techs: " + faction,
                  xaxis_title="Average Score",
                  yaxis_title="Advanced Tech")

    fig.write_html("./plots/adv_techs_analysis/pos_on_score/" + faction + ".html")
    fig.show()


5156